In [ ]:
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.preprocessing import image

# Build the feature extractor
def build_model():
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    model = Sequential([
        base_model,
        GlobalMaxPooling2D()
    ])
    return model

model = build_model()


In [ ]:
from tensorflow.keras.preprocessing import image
from sklearn.preprocessing import normalize
from PIL import Image
import numpy as np

def extract_feature(img_path, model) -> np.ndarray:
    try:
        img = Image.open(img_path).convert('RGB').resize((224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)

        features = model.predict(img_array)
        features = features.flatten()

        normalized = normalize([features])[0]
        return normalized
    except Exception as e:
        print(f"[ERROR] Failed to process {img_path}: {e}")
        return None


In [ ]:
import os
import numpy as np
import pickle
from tqdm import tqdm

model = build_model()
IMG_DIR = "wardrobe_images/wardrobe_images"
FEATURES_DIR = "features"
os.makedirs(FEATURES_DIR, exist_ok=True)

feature_map = {}

for category in os.listdir(IMG_DIR):
    category_path = os.path.join(IMG_DIR, category)
    if not os.path.isdir(category_path):
        continue

    for img_name in tqdm(os.listdir(category_path), desc=category):
        img_path = os.path.join(category_path, img_name)
        features = extract_feature(img_path, model)

        if features is not None:
            feature_map[img_path] = features

# Save all features
with open(os.path.join(FEATURES_DIR, "features.pkl"), "wb") as f:
    pickle.dump(feature_map, f)


In [ ]:
def parse_tags(filename: str):
    name = os.path.basename(filename).lower()
    parts = name.split("_")
    if len(parts) >= 4:
        return {
            "color": parts[1],
            "type": parts[2],
            "occasion": parts[3]
        }
    return {}


In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pickle

# Load features
with open("features/features.pkl", "rb") as f:
    features_dict = pickle.load(f)

paths = list(features_dict.keys())
features = np.array(list(features_dict.values()))

# Build KNN model
knn = NearestNeighbors(n_neighbors=5, metric='cosine')
knn.fit(features)

def recommend_similar(image_path, model):
    feat = extract_feature(image_path, model)
    distances, indices = knn.kneighbors([feat])
    return [paths[i] for i in indices[0]]


In [ ]:
def recommend_by_occasion(image_path, model, occasion, metadata):
    filtered = [(p, f) for p, f in metadata.items() if parse_tags(p).get("occasion") == occasion]
    if not filtered:
        return []
    
    paths, feats = zip(*filtered)
    knn = NearestNeighbors(n_neighbors=5, metric='cosine').fit(feats)

    feat = extract_feature(image_path, model)
    distances, indices = knn.kneighbors([feat])
    return [paths[i] for i in indices[0]]
